In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
cd /kaggle/input/mobilefull/

/kaggle/input/mobilefull


In [3]:
pwd

'/kaggle/input/mobilefull'

In [4]:
from model import DepthEstimate
model = DepthEstimate()
batch_size     = 16

learning_rate  = 0.0001
epochs         = 6

/opt/conda/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 2s 0us/step
Base model loaded

Model created.


In [5]:
import tensorflow as tf

from io import BytesIO
from zipfile import ZipFile
from sklearn.utils import shuffle

class DataLoader():
    def __init__(self, csv_file='data/nyu2_train.csv', DEBUG=False):
        self.shape_rgb = (480,640, 3)
        self.shape_depth = (240,320, 1)
        self.read_nyu_data(csv_file, DEBUG=DEBUG)

    def nyu_resize(self, img, resolution=224, padding=6):
        from skimage.transform import resize
        return resize(img, (resolution, resolution), preserve_range=True, mode='reflect', anti_aliasing=True )

    def read_nyu_data(self, csv_file, DEBUG=False):
        csv = open(csv_file, 'r').read()
        nyu2_train = list((row.split(',') for row in (csv).split('\n') if len(row) > 0))

        # Dataset shuffling happens here
        nyu2_train = shuffle(nyu2_train, random_state=0)

        # Test on a smaller dataset
        if DEBUG: nyu2_train = nyu2_train[:10]
        
        # A vector of RGB filenames.
        self.filenames = [i[0] for i in nyu2_train]

        # A vector of depth filenames.
        self.labels = [i[1] for i in nyu2_train]

        # Length of dataset
        self.length = len(self.filenames)

    def _parse_function(self, filename, label): 
        # Read images from disk
        #image_decoded = tf.image.decode_jpeg(tf.io.read_file(filename))
        depth_resized = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(label)), [self.shape_depth[0], self.shape_depth[1]])
        image_decoded = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(filename)), [self.shape_rgb[0], self.shape_rgb[1]])
        # Format
        rgb = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        depth = tf.image.convert_image_dtype(depth_resized / 255.0, dtype=tf.float32)
        
        # Normalize the depth values (in cm)
        depth = 1000 / tf.clip_by_value(depth * 1000, 10, 1000)

        return rgb, depth

    def get_batched_dataset(self, batch_size):
        self.dataset = tf.data.Dataset.from_tensor_slices((self.filenames, self.labels))
        self.dataset = self.dataset.shuffle(buffer_size=len(self.filenames), reshuffle_each_iteration=True)
        self.dataset = self.dataset.repeat()
        self.dataset = self.dataset.map(map_func=self._parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        #self.dataset = self.dataset.map(map_func=self.nyu_resize)
        self.dataset = self.dataset.batch(batch_size=batch_size)

        return self.dataset

In [6]:
cd /kaggle/input/extract

/kaggle/input/extract


In [7]:
dl = DataLoader()
train_generator = dl.get_batched_dataset(batch_size)

print('Data loader ready.')

Data loader ready.


In [8]:
import tensorflow.keras.backend as K

def depth_loss_function(y_true, y_pred, theta=0.1, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Edges
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w2 = 1.0
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

In [9]:
import tensorflow
#from loss import depth_loss_function

optimizer = tensorflow.keras.optimizers.Adam(lr=learning_rate, amsgrad=True)

model.compile(loss=depth_loss_function, optimizer=optimizer)

In [10]:
cd /kaggle/input/test-data

/kaggle/input/test-data


In [11]:
import numpy as np
from io import BytesIO
rgb = np.load('eigen_test_rgb.npy')
depth = np.load('eigen_test_depth.npy')
crop = np.load('eigen_test_crop.npy')
print('Test data loaded.\n')
print(np.shape(rgb))
print(np.shape(depth))

Test data loaded.

(654, 480, 640, 3)
(654, 480, 640)


In [12]:
cd /kaggle/input/evaluateutils


/kaggle/input/evaluateutils


In [13]:
from utils import predict, load_images, display_images, evaluate
class ev_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    b=time.time()
    e = evaluate(model, rgb, depth, crop, batch_size=1)
    print("eval results ")
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))
    print("eval time...")
    print(time.time()-b)

In [14]:
cd /kaggle/input/extract

/kaggle/input/extract


In [15]:
import os
import tensorflow
checkpoint_path = "/kaggle/working/training_mob_bs16/cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
import time
a=time.time()
history=model.fit(train_generator, epochs=epochs, steps_per_epoch=dl.length//batch_size, callbacks=[cp_callback,ev_callback()],initial_epoch=0)

Train for 3168 steps
Epoch 1/6
3167/3168 [============================>.] - ETA: 0s - loss: 0.1682
Epoch 00001: saving model to /kaggle/working/training_mob_bs16/cp.ckpt
eval results 
        a1,         a2,         a3,        rel,        rms,     log_10
    0.2811,     0.5434,     0.7586,     0.6576,     1.6815,     0.2015
eval time...
162.74237823486328
3168/3168 [==============================] - 2761s 871ms/step - loss: 0.1682
Epoch 2/6
3167/3168 [============================>.] - ETA: 0s - loss: 0.1139
Epoch 00002: saving model to /kaggle/working/training_mob_bs16/cp.ckpt
eval results 
        a1,         a2,         a3,        rel,        rms,     log_10
    0.3185,     0.6243,     0.8197,     0.5413,     1.5616,     0.1777
eval time...
148.99979305267334
3168/3168 [==============================] - 2702s 853ms/step - loss: 0.1139
Epoch 3/6
3167/3168 [============================>.] - ETA: 0s - loss: 0.1014
Epoch 00003: saving model to /kaggle/working/training_mob_bs16/cp.ckpt
ev